In [1]:
import pathlib
import sys

sys.path.append(str(pathlib.Path().absolute().parent))

from src.replay_memory import FastReplayBuffer
from src.env import FourRoomEnvWithTagging


In [2]:
BUF_SIZE = 3000
N_IMPOSTERS = 2
N_JOBS = 5
N_CREW = 4
SEQUENCE_SIZE = 2
DEBUG = True

env = FourRoomEnvWithTagging(n_imposters=N_IMPOSTERS, n_crew=N_CREW, n_jobs=N_JOBS, debug=DEBUG)
m = FastReplayBuffer(max_size=BUF_SIZE, state_size=env.flattened_state_size, trajectory_size=SEQUENCE_SIZE, n_agents=env.n_agents, n_imposters=N_IMPOSTERS)
m.populate(env, 1000)

DEBUG - 
New Game Started!
-----------------
    Agent Positions: [(1, 5), (2, 3), (7, 3), (6, 1), (5, 0), (1, 8)]
    Imposters: [1 2]
    Crew Members: [0 3 4 5]
    Alive Agents: [ True  True  True  True  True  True]
    Job Positions: [(3, 2), (3, 8), (7, 8), (8, 3), (2, 0)]
    Completed Jobs: [False False False False False]
    Tag Counts: [0. 0. 0. 0. 0. 0.]
    Time Left for Tag Reset: 50
    Used Tag Actions: [0. 0. 0. 0. 0. 0.]
-----------------
        
DEBUG - Agent 2 ([7 3]) tagged Agent 5 ([1 8])! 5's new tag count: 1.0
DEBUG - Agent 0 ([1 5]) tagged Agent 4 ([5 0])! 4's new tag count: 1.0
DEBUG - Agent 4 ([5 0]) tagged Agent 3 ([7 1])! 3's new tag count: 1.0
DEBUG - Agent 0 tried to tag Agent 2 but failed!
DEBUG - Agent 2 tried to tag Agent 3 but failed!
DEBUG - Agent 5 ([1 7]) tagged Agent 0 ([1 5])! 0's new tag count: 1.0
DEBUG - Agent 3 ([7 1]) tagged Agent 4 ([5 0])! 4's new tag count: 2.0
DEBUG - Agent 1 ([2 2]) tagged Agent 5 ([1 7])! 5's new tag count: 2.0
DEBUG -

In [3]:
env.state_fields

{<StateFields.AGENT_POSITIONS: 0>: 0,
 <StateFields.JOB_POSITIONS: 1>: 1,
 <StateFields.JOB_STATUS: 2>: 2,
 <StateFields.ALIVE_AGENTS: 3>: 3,
 <StateFields.USED_TAGS: 4>: 4,
 <StateFields.TAG_COUNTS: 5>: 5,
 <StateFields.TAG_RESET_COUNT: 6>: 6}

In [4]:
from src.featurizers import SequenceStateFeaturizer
from src.visualize import SequenceStateVisualizer

batch = m.sample(1)

seq_states = batch.states.squeeze(0)
seq_imposters = batch.imposters.squeeze(0)

f = SequenceStateFeaturizer(env, seq_states, seq_imposters)
v = SequenceStateVisualizer(f)
# v.visualize_perspectives()

(array([[1, 6],
       [1, 2],
       [5, 3],
       [7, 3],
       [6, 1],
       [1, 8]]), array([[3, 2],
       [3, 8],
       [7, 8],
       [8, 3],
       [2, 0]]), array([0, 0, 0, 1, 0], dtype=int8), array([1, 1, 1, 0, 1, 1], dtype=int8), array([1, 1, 1, 1, 1, 1], dtype=int8), array([1, 0, 0, 0, 2, 2]), array([16]))
(array([[1, 6],
       [1, 2],
       [5, 3],
       [7, 3],
       [6, 1],
       [1, 8]]), array([[3, 2],
       [3, 8],
       [7, 8],
       [8, 3],
       [2, 0]]), array([0, 0, 0, 1, 0], dtype=int8), array([1, 1, 1, 0, 1, 1], dtype=int8), array([1, 1, 1, 1, 1, 1], dtype=int8), array([1, 0, 0, 0, 2, 2]), array([15]))
